# Hamilton matrices for spin models

Consider a spin model of the form
\begin{equation}
    H = -\sum\limits_{ij\alpha} J_{ij}^\alpha S_i^\alpha S_j^\alpha
\end{equation}
with $i,j=1,\dots, N$ and $\alpha=x,y,z$.

## a) Implementation

In [174]:
using SparseArrays
using LinearAlgebra
using Latexify
using SymPy

In [285]:
N = 3

function reset_Js(N) 
    Js = [sparse(zeros(N,N)) for α in 1:3]; # 3 N×N matrices (α = x,y,z)
    return Js
end

Js = reset_Js(N);

# initialize Js with values here
# ...

In [283]:
# convert [n]₁₀ to [n]₂ (in reversed order of notation, that is also used for |n⟩)
n_dual(n) = digits(n, base=2, pad=N)

# Function to calculate |n⟩ ↦ |l⟩ for the action of SᵢSⱼ for α = x,y
l(n,i,j) = n + (1 - 2 * n_dual(n)[i]) * 2^(i-1) + (1 - 2 * n_dual(n)[j]) * 2^(j-1)

l (generic function with 1 method)

In [284]:
# prefactor λˣ for the x-component of the interaction of spins i and j of |n⟩ 
#    -JˣᵢⱼSˣᵢSˣⱼ|n⟩ ↦ λʸ|l⟩
function x_link_prefactor(Js, n, i, j)
    α = 1 # x =̂ 1
    return -1//4 * Js[α][i,j]
end

# prefactor λʸ for the y-component of the interaction of spins i and j of |n⟩ 
#    -JʸᵢⱼSʸᵢSʸⱼ|n⟩ ↦ λʸ|l⟩
function y_link_prefactor(Js, n, i, j)
    α = 2 # y =̂ 2
    # calculate the sign, +1 for nᵢ != nⱼ, -1 for nᵢ == nⱼ
    s = - (2 * n_dual(n)[i] - 1) * (2 * n_dual(n)[j] - 1)
    return - 1//4 * s * Js[α][i,j]
end

# prefactor λᶻ for the z-component of the interaction of spins i and j of |n⟩ 
#    -JᶻᵢⱼSᶻᵢSᶻⱼ|n⟩ ↦ λᶻ|l⟩
function z_link_prefactor(Js, n, i, j)
    α = 3 # z =̂ 3
    # calculate the sign, +1 for nᵢ == nⱼ, -1 for nᵢ != nⱼ
    s = (2 * n_dual(n)[i] - 1) * (2 * n_dual(n)[j] - 1)
    return -1//4 * s * Js[α][i,j]
end

z_link_prefactor (generic function with 2 methods)

In [287]:
function calculate_hamilton_matrix(Js, N)
    H̄ = zeros(2^N, 2^N) # empty 2ᴺ×2ᴺ matrix
    for n in 0:(2^N-1)
        # for each |n⟩ set the non-zero matrix elements
        
        # 1. loop over all components α = x,y,z 
        for α in [1,2,3]
            # 2. loop over only non-zero links in Js
            # iterate over non-zero elements by using findnz()
            for (i, j, J) in zip(findnz(Js[α])...)
                
                if α == 1 # α =̂ x
                    m = l(n, i, j) # calculate |l⟩ from |n⟩ for i,j
                    # add to the matrix element ⟨l|H|n⟩ the x-link-term
                    H̄[m+1,n+1] += x_link_prefactor(Js, n, i, j)
                elseif α == 2 # α =̂ y
                    m = l(n, i, j) # calculate |l⟩ from |n⟩ for i,j
                    # add to the matrix element ⟨l|H|n⟩ the y-link-term
                    H̄[m+1,n+1] += y_link_prefactor(Js, n, i, j)
                else
                    # α =̂ z, diagonal element
                    H̄[n+1,n+1] += z_link_prefactor(Js, n, i, j)
                end
            end
        end
    end
    return H̄
end

calculate_hamilton_matrix (generic function with 1 method)

## b) Three-site clusters
### Ising-model
$J_{ij}^\alpha = J\delta_{\alpha, z}$ and $i < j$

In [292]:
N = 3

# Ising
Js_ising = reset_Js(N)
# 1 (J) for all i<j for α =̂ z, 0 for α ≠ z
Js_ising[3] = [
    0 1 1
    0 0 1
    0 0 0
]

H̄_ising = calculate_hamilton_matrix(Js_ising, N)

8×8 Matrix{Float64}:
 -0.75  0.0   0.0   0.0   0.0   0.0   0.0    0.0
  0.0   0.25  0.0   0.0   0.0   0.0   0.0    0.0
  0.0   0.0   0.25  0.0   0.0   0.0   0.0    0.0
  0.0   0.0   0.0   0.25  0.0   0.0   0.0    0.0
  0.0   0.0   0.0   0.0   0.25  0.0   0.0    0.0
  0.0   0.0   0.0   0.0   0.0   0.25  0.0    0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.25   0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   -0.75

In [293]:
rationalize.(H̄_ising) * sympify("J")

8×8 Matrix{Sym}:
 -3*J/4    0    0    0    0    0    0       0
      0  J/4    0    0    0    0    0       0
      0    0  J/4    0    0    0    0       0
      0    0    0  J/4    0    0    0       0
      0    0    0    0  J/4    0    0       0
      0    0    0    0    0  J/4    0       0
      0    0    0    0    0    0  J/4       0
      0    0    0    0    0    0    0  -3*J/4

#### Eigenenergies

In [444]:
E_ising = eigvals(rationalize.(H̄_ising) * sympify("J"))

2-element Vector{Sym}:
 -3*J/4
    J/4

### Isotropic Heisenberg
$J_{ij}^\alpha = J$ and $i<j$

In [295]:
N = 3

# Isotropic Heisenberg
Js_Heisenberg = reset_Js(N)
# 1 (J) for all i<j for α =̂ z, 0 for α ≠ z
Js_Heisenberg[1] = [
    0 1 1
    0 0 1
    0 0 0
]
Js_Heisenberg[2] = Js_Heisenberg[1]
Js_Heisenberg[3] = Js_Heisenberg[1]

H̄_Heisenberg = calculate_hamilton_matrix(Js_Heisenberg, N)

8×8 Matrix{Float64}:
 -0.75   0.0    0.0    0.0    0.0    0.0    0.0    0.0
  0.0    0.25  -0.5    0.0   -0.5    0.0    0.0    0.0
  0.0   -0.5    0.25   0.0   -0.5    0.0    0.0    0.0
  0.0    0.0    0.0    0.25   0.0   -0.5   -0.5    0.0
  0.0   -0.5   -0.5    0.0    0.25   0.0    0.0    0.0
  0.0    0.0    0.0   -0.5    0.0    0.25  -0.5    0.0
  0.0    0.0    0.0   -0.5    0.0   -0.5    0.25   0.0
  0.0    0.0    0.0    0.0    0.0    0.0    0.0   -0.75

In [296]:
rationalize.(H̄_Heisenberg) * sympify("J")

8×8 Matrix{Sym}:
 -3*J/4     0     0     0     0     0     0       0
      0   J/4  -J/2     0  -J/2     0     0       0
      0  -J/2   J/4     0  -J/2     0     0       0
      0     0     0   J/4     0  -J/2  -J/2       0
      0  -J/2  -J/2     0   J/4     0     0       0
      0     0     0  -J/2     0   J/4  -J/2       0
      0     0     0  -J/2     0  -J/2   J/4       0
      0     0     0     0     0     0     0  -3*J/4

#### Eigenenergies

In [445]:
E_Heisenberg = eigvals(rationalize.(H̄_Heisenberg) * sympify("J"))

2-element Vector{Sym}:
 -3*J/4
  3*J/4

### model 1d
$J_{12}^x = J_{23}^y = J_{13}^z = J$ and all other $J_{ij}^\alpha = 0$ and $i<j$

In [297]:
N = 3

# Isotropic Heisenberg
Js_1d = reset_Js(N)
# 1 (J) for all (1,2,x), (2,3,y), (1,3,z)
Js_1d[1][1,2] = 1
Js_1d[2][2,3] = 1
Js_1d[3][1,3] = 1
H̄_1d = calculate_hamilton_matrix(Js_1d, N)

8×8 Matrix{Float64}:
 -0.25   0.0    0.0   -0.25   0.0    0.0    0.25   0.0
  0.0    0.25  -0.25   0.0    0.0    0.0    0.0    0.25
  0.0   -0.25  -0.25   0.0   -0.25   0.0    0.0    0.0
 -0.25   0.0    0.0    0.25   0.0   -0.25   0.0    0.0
  0.0    0.0   -0.25   0.0    0.25   0.0    0.0   -0.25
  0.0    0.0    0.0   -0.25   0.0   -0.25  -0.25   0.0
  0.25   0.0    0.0    0.0    0.0   -0.25   0.25   0.0
  0.0    0.25   0.0    0.0   -0.25   0.0    0.0   -0.25

In [298]:
rationalize.(H̄_1d) * sympify("J")

8×8 Matrix{Sym}:
 -J/4     0     0  -J/4     0     0   J/4     0
    0   J/4  -J/4     0     0     0     0   J/4
    0  -J/4  -J/4     0  -J/4     0     0     0
 -J/4     0     0   J/4     0  -J/4     0     0
    0     0  -J/4     0   J/4     0     0  -J/4
    0     0     0  -J/4     0  -J/4  -J/4     0
  J/4     0     0     0     0  -J/4   J/4     0
    0   J/4     0     0  -J/4     0     0  -J/4

In [446]:
E_1d = eigvals(rationalize.(H̄_1d) * sympify("J"))

2-element Vector{Sym}:
 -sqrt(3)*J/4
  sqrt(3)*J/4

search: eigen eigen! Eigen GeneralizedEigen eigvecs leading_zeros leading_ones



```
eigen(A; permute::Bool=true, scale::Bool=true, sortby) -> Eigen
```

Computes the eigenvalue decomposition of `A`, returning an [`Eigen`](@ref) factorization object `F` which contains the eigenvalues in `F.values` and the eigenvectors in the columns of the matrix `F.vectors`. (The `k`th eigenvector can be obtained from the slice `F.vectors[:, k]`.)

Iterating the decomposition produces the components `F.values` and `F.vectors`.

The following functions are available for `Eigen` objects: [`inv`](@ref), [`det`](@ref), and [`isposdef`](@ref).

For general nonsymmetric matrices it is possible to specify how the matrix is balanced before the eigenvector calculation. The option `permute=true` permutes the matrix to become closer to upper triangular, and `scale=true` scales the matrix by its diagonal elements to make rows and columns more equal in norm. The default is `true` for both options.

By default, the eigenvalues and vectors are sorted lexicographically by `(real(λ),imag(λ))`. A different comparison function `by(λ)` can be passed to `sortby`, or you can pass `sortby=nothing` to leave the eigenvalues in an arbitrary order.   Some special matrix types (e.g. [`Diagonal`](@ref) or [`SymTridiagonal`](@ref)) may implement their own sorting convention and not accept a `sortby` keyword.

# Examples

```jldoctest
julia> F = eigen([1.0 0.0 0.0; 0.0 3.0 0.0; 0.0 0.0 18.0])
Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
  1.0
  3.0
 18.0
vectors:
3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

julia> F.values
3-element Vector{Float64}:
  1.0
  3.0
 18.0

julia> F.vectors
3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

julia> vals, vecs = F; # destructuring via iteration

julia> vals == F.values && vecs == F.vectors
true
```

---

```
eigen(A, B) -> GeneralizedEigen
```

Computes the generalized eigenvalue decomposition of `A` and `B`, returning a [`GeneralizedEigen`](@ref) factorization object `F` which contains the generalized eigenvalues in `F.values` and the generalized eigenvectors in the columns of the matrix `F.vectors`. (The `k`th generalized eigenvector can be obtained from the slice `F.vectors[:, k]`.)

Iterating the decomposition produces the components `F.values` and `F.vectors`.

Any keyword arguments passed to `eigen` are passed through to the lower-level [`eigen!`](@ref) function.

# Examples

```jldoctest
julia> A = [1 0; 0 -1]
2×2 Matrix{Int64}:
 1   0
 0  -1

julia> B = [0 1; 1 0]
2×2 Matrix{Int64}:
 0  1
 1  0

julia> F = eigen(A, B);

julia> F.values
2-element Vector{ComplexF64}:
 0.0 - 1.0im
 0.0 + 1.0im

julia> F.vectors
2×2 Matrix{ComplexF64}:
  0.0+1.0im   0.0-1.0im
 -1.0+0.0im  -1.0-0.0im

julia> vals, vecs = F; # destructuring via iteration

julia> vals == F.values && vecs == F.vectors
true
```

---

```
eigen(A::Union{SymTridiagonal, Hermitian, Symmetric}, irange::UnitRange) -> Eigen
```

Computes the eigenvalue decomposition of `A`, returning an [`Eigen`](@ref) factorization object `F` which contains the eigenvalues in `F.values` and the eigenvectors in the columns of the matrix `F.vectors`. (The `k`th eigenvector can be obtained from the slice `F.vectors[:, k]`.)

Iterating the decomposition produces the components `F.values` and `F.vectors`.

The following functions are available for `Eigen` objects: [`inv`](@ref), [`det`](@ref), and [`isposdef`](@ref).

The [`UnitRange`](@ref) `irange` specifies indices of the sorted eigenvalues to search for.

!!! note
    If `irange` is not `1:n`, where `n` is the dimension of `A`, then the returned factorization will be a *truncated* factorization.


---

```
eigen(A::Union{SymTridiagonal, Hermitian, Symmetric}, vl::Real, vu::Real) -> Eigen
```

Computes the eigenvalue decomposition of `A`, returning an [`Eigen`](@ref) factorization object `F` which contains the eigenvalues in `F.values` and the eigenvectors in the columns of the matrix `F.vectors`. (The `k`th eigenvector can be obtained from the slice `F.vectors[:, k]`.)

Iterating the decomposition produces the components `F.values` and `F.vectors`.

The following functions are available for `Eigen` objects: [`inv`](@ref), [`det`](@ref), and [`isposdef`](@ref).

`vl` is the lower bound of the window of eigenvalues to search for, and `vu` is the upper bound.

!!! note
    If [`vl`, `vu`] does not contain all eigenvalues of `A`, then the returned factorization will be a *truncated* factorization.

